# دانلود فایل تلگرام و ذخیره مستقیم در Google Drive (بدون message_id)

این نوت‌بوک کل مسیر را برای کاربر غیر فنی ساده می‌کند: دریافت پروژه، نصب، ورود به تلگرام، انتظار برای دریافت فایل، و ذخیره در گوگل درایو.

## 1) دریافت خودکار پروژه از GitHub
این سلول سورس پروژه را دریافت می‌کند.

In [ ]:
!rm -rf /content/telegram-download-daemon
!git clone -q https://github.com/hamidrg20001379/telegram-download-daemon /content/telegram-download-daemon
%cd /content/telegram-download-daemon

## 2) نصب وابستگی‌ها
ابتدا وابستگی‌های پروژه و ابزارهای لازم برای Colab نصب می‌شوند.

In [ ]:
!pip -q install -r /content/telegram-download-daemon/requirements.txt
!pip -q install nest_asyncio

## 3) اتصال Google Drive
بعد از اجرا، روی لینک کلیک کنید و مجوز دسترسی را بدهید.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 4) تنظیمات ساده کاربر
- `api_id` و `api_hash` را از https://core.telegram.org/api/obtaining_api_id بگیرید.
- برای کاربران عادی، `source_chat='me'` بگذارید و فایل را به **Saved Messages** بفرستید.

In [ ]:
api_id = int(input('API ID: '))
api_hash = input('API HASH: ').strip()
phone = input('Phone (example: +98912...): ').strip()
source_chat = input("Source chat (default: me): ").strip() or 'me'
drive_folder = input('Drive folder (default: TelegramDownloads): ').strip() or 'TelegramDownloads'
download_dir = f'/content/drive/MyDrive/{drive_folder}'

## 5) ورود به تلگرام و دانلود خودکار اولین فایل جدید
بعد از اجرای این سلول:
1. در تلگرام یک فایل به چت انتخابی (معمولاً Saved Messages) بفرستید یا Forward کنید.
2. نوت‌بوک به‌صورت خودکار فایل جدید را پیدا و در Google Drive ذخیره می‌کند.

In [ ]:
import os
import asyncio
import nest_asyncio
from getpass import getpass
from telethon import TelegramClient
from telethon.errors import SessionPasswordNeededError

nest_asyncio.apply()
os.makedirs(download_dir, exist_ok=True)

async def wait_for_new_media(client, chat, after_id, poll_seconds=3, timeout_seconds=1800):
    waited = 0
    while waited < timeout_seconds:
        msgs = await client.get_messages(chat, limit=10)
        candidates = [m for m in msgs if m and m.id and m.id > after_id and m.media]
        if candidates:
            return max(candidates, key=lambda m: m.id)
        await asyncio.sleep(poll_seconds)
        waited += poll_seconds
    raise TimeoutError('در زمان تعیین‌شده فایل جدیدی دریافت نشد.')

async def main():
    client = TelegramClient('colab_session', api_id, api_hash)
    await client.connect()

    if not await client.is_user_authorized():
        await client.send_code_request(phone)
        code = input('کد ارسال‌شده توسط تلگرام را وارد کنید: ').strip()
        try:
            await client.sign_in(phone=phone, code=code)
        except SessionPasswordNeededError:
            password = getpass('پسورد تایید دومرحله‌ای: ')
            await client.sign_in(password=password)

    latest = await client.get_messages(source_chat, limit=1)
    baseline_id = latest[0].id if latest else 0

    print('✅ ورود موفق بود.')
    print('⏳ حالا فایل را به چت انتخابی بفرستید/فوروارد کنید...')
    print(f'📥 مقصد ذخیره‌سازی: {download_dir}')

    msg = await wait_for_new_media(client, source_chat, baseline_id)
    saved_path = await client.download_media(msg, file=download_dir)
    print(f'✅ دانلود کامل شد. مسیر فایل: {saved_path}')

    await client.disconnect()

await main()

## 6) نتیجه
فایل دانلودی در مسیر `MyDrive/<نام پوشه انتخابی شما>` قرار می‌گیرد.

اگر خواستید فایل دیگری ذخیره کنید، فقط سلول مرحله 5 را دوباره اجرا کنید.